In [ ]:
#pragma cling add_include_path("/p/project/training2312/local/include")

In [ ]:
#include <iostream>
#include <cmath>

User defined data types are created using the `struct` or `class` keyword. (They are more or less the same. The minor differences will be pointed out later.)

In [ ]:
class A{};
class B{};

The above are the smallest possible classes in C++, not containing anything in them. But they are still **different types**. Objects of type `A` are considered to be of a different unrelated type compared to objects of type `B`. Type identity is not based on what is inside a class. Two type names would be regarded as the same only if one is a type alias of another. Type aliases are created using the keyword `using`:

In [ ]:
using C = A;

In [ ]:
void f(int i, A a)
{
    std::cout << "Version of f overloaded for int and A\n";
}

In [ ]:
void f(int i, B a)
{
    std::cout << "Version of f overloaded for int and B\n";
}

In [ ]:
{
    int num{88};
    A xa;
    B xb;
    C xc;
    f(num, xa);
    f(num, xb);
    f(num, xc);
}

A user defined data type can optionally define "member" fields.

In [ ]:
struct Example1 {
    int x{}, y{};
};

Each "instance" of an object of such a type has each of the data fields defined in the `class`. The members (components) of an instance are accessed using a dot notation for references and `->` notation for pointers.

In [ ]:
{
    Example1 v1{2, 4};
    Example1 v2{98, 3};
    std::cout << "Components of v1 : " << v1.x << ", " << v1.y << "\n";
    std::cout << "Components of v2 : " << v2.x << ", " << v2.y << "\n";
    std::cout << "Size of an object of type Example1 = " << sizeof(Example1) << " bytes\n";
    Example1* vp{&v1};
    std::cout << "Components of the entity vp is pointing to : " << vp->x << ", " << vp->y << "\n";
}

Classes are supposed to represent conceptual types : think complex numbers, 3D vectors, Atoms, Molecules, distances, ... Nouns we need to describe our problem are candidates to be classes. We model such entities by filling the classes with the right kind of members (properties). These members could be function members as well...

In [ ]:
struct Example2 {
    int x{}, y{};
    auto mod() -> int { return x * x + y * y; }
};

In [ ]:
{
    Example2 v1{2, 4};
    Example2 v2{98, 3};
    std::cout << "Components of v1 : " << v1.x << ", " << v1.y << "\n";
    std::cout << "Components of v2 : " << v2.x << ", " << v2.y << "\n";
    std::cout << "Size of an object of type Example2 = " << sizeof(Example2) << " bytes\n";
    std::cout << "mod result from v1 = " << v1.mod() << "\n";
    std::cout << "mod result from v2 = " << v2.mod() << "\n";
    Example2* vp{&v1};
    std::cout << "mod of the entity vp is point to = " << vp->mod() << "\n";
}

Member functions are accessed using the same notation as data members (dot or `->`). 

Only data members contribute to the size of one instance of a class. An object of a class with two `int` members and 7000 member functions will still take 8 bytes to store.

Member functions are defined once for each class. They are not attached to an instance. When we call `v1.mod()`, conceptually it is like `Example2::mod(&v1)`. We can't ourselves write it like this, but the compiler does something similar. The member functions are translated with a hidden first argument named `this`, as if the function was : 

```c++
// only a conceptual model
auto Example2_conceptual_mod(Example* this) -> int
{
    return this->x * this->x + this->y * this->y;
}
```

The `this` pointer is real. It is available for use inside member functions in real code, akin to `self` in python. But in C++, `this->x` inside a member function can be shortened to `x`. This `this` pointer does not take any space inside the class: it is the address of the instance on the left of the dot, so that the compiler can always forward it correctly for every call to a member function. 

In C++, the composition of  a class in terms of its data and function members is a compile time constant. Properties of a class can not be added, removed or changed outside the class definition. When we see a variable (instance) of a certain type (class), we know exactly what properties it has, how many bytes it occupies etc., without considering the history of that variable in the program. One specific complex number can not in the course of the program execution acquire a new third component! All instances have the same data fields, and the same set of member functions. This enables regular, predictable and controllable behaviour when we use class type objects in regular code. 

## const member functions

How would we indicate that the function `mod` in the `Example2` class above does not change any data members ? In the conceptual model pseudo-implementation as a free-standing function, we could do that easily:

```c++
auto Example2_conceptual_mod(const Example2* this) -> int
{
    return this->x * this->x + this->y * this->y;
}
```

This would be possible because we have explicit access to the formal parameter, and we can make it `const`. But in the real code, the `this` pointer is passed implicitly. But we need some way to indicate to the compiler that it should generate the implicitly passed `this` pointer of the type `const Example2*` rather than `Example2*`. This is accomplished using the following notation:

```c++
struct Example3 {
    int x{}, y{};
    auto mod() const -> int { return x * x + y * y; }
};
```
With that, any attempt to change member variables in `mod()` would result in a compiler error.

In [ ]:
struct Example3 {
    int x{}, y{};
    auto mod() const -> int 
    {
        // careless change of member...
        x = x + y;
        return x * x + y * y; 
    }
};

## Operator overloading

For user defined types, we can define the meaning of operators such as `+`, `-`, `*`, `/`, `%`, `<<` ... For instance, we can define addition of two objects of the type `Example2` as follows:

```c++
auto operator+(const Example2& e1, const Example2& e2) -> Example2
{
    return {e1.x + e2.x, e1.y + e2.y};
}
```
However, this is incorrectly not accepted by the jupyter-cling environment. Making the operator into a member function allows this to be executed in this interpreter. In real life code, the free standing operator above, being more symmetric between the two operands, is more elegant and preferable. For now, let's see how operator overloading looks like in practice...

In [ ]:
struct Example4 {
    int x{}, y{};
    auto mod() -> int { return x * x + y * y; }
    auto operator+(const Example4& other) const -> Example4
    {
        return {x + other.x, y + other.y };
    }
};

In [ ]:
{
    Example4 a{10, 20}, b{3, 4};
    auto c = a + b;
    std::cout << "c = [" << c.x << ", " << c.y << "]\n";
}

Being able to overload operators like this allows our class types to behave much like built in types. The overloaded operators need one argument from our class type. Many operators can be overloaded. Most versatile among them is perhaps the function call operator `()`. We can write a class, for which we have an overloaded `()` operator. That allows objects of that class to be used as if they were functions. Such entities are variously called functionals, function objects, function-like objects ...

In [ ]:
struct SineWave {
    double amp{1.0}, ome{1.0}, pha{0.0};
    auto operator()(double t) const noexcept -> double
    {
        return amp * sin(ome * t + pha);
    }
};

Notice how the interpreter accepts a trailing return type for the function in this case!

We can create objects of this type, and use them as if they were functions...


In [ ]:
SineWave S1{0.5, 0.1, 0.3}, S2{3.0, 1.4, 0.8}, S3{2.0, 1.0, 1.8};
for (double t=0.; t < 2.0; t += 0.05)
    std::cout << t << "\t" << S1(t) << "\t" << S2(t) << "\t" << S3(t) << "\n";